In [6]:
import pandas as pd
from datetime import timedelta
from DAX.HelpFunctions.get_dax_data import get_dax_data
import numpy as np
from HelpFunctions.date_and_time import most_recent_thursday
from HelpFunctions.date_and_time import next_working_days
import statsmodels.api as sm
pd.set_option('display.width', 400)

In [2]:
import pandas as pd

# Example DataFrame
data = {'Category': ['A', 'B', 'A', 'C', 'B', 'A', 'B', 'A', 'C', 'B', 'A', 'C'],
        'Value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
df = pd.DataFrame(data)

# Group by 'Category' and filter entries with a count > 5
print(df.groupby('Category')['Category'].transform('count'))
grouped = df.groupby('Category')['Category'].transform('count')
mask = grouped > 5
filtered_df = df[mask]

print(filtered_df)


0     5
1     4
2     5
3     3
4     4
5     5
6     4
7     5
8     3
9     4
10    5
11    3
Name: Category, dtype: int64
Empty DataFrame
Columns: [Category, Value]
Index: []


In [7]:
df = get_dax_data()
df = df[['ret1', 'ret2', 'ret3', 'ret4', 'ret5']]
df = df.dropna()

In [8]:
start_date_excl = most_recent_thursday(df) - timedelta(days=1)
df = df.loc[df.index < start_date_excl]
df.index = df.index.date

In [9]:
def rolling_ret_volatility(ret, days=5):
    volatility_measure = []
    for i in range(len(ret)):
        start_index = max(0, i-days+1)
        end_index = i + 1
        
        vol = ret.iloc[start_index:end_index].abs().sum()
        
        volatility_measure.append(vol)
    return volatility_measure


def quant_reg(df):    
    df = df.dropna()
    df = df[-1000:]
    df['volatility'] = rolling_ret_volatility(df['ret1'])
    
    horizons = np.arange(1,6)
    
    forecasts = []
    for horizon in horizons:
        ret = df[f'ret{horizon}']
        vol = df['volatility']
        forecasts.append(_quant_reg_one_horizon(ret, vol, horizon))
    forecasts = pd.concat(forecasts)
    print(forecasts)
        
    


def _quant_reg_one_horizon(ret, vol, horizon):
    df = pd.DataFrame()
    df['ret'] = ret
    df['volatility'] = vol    
    
    y = df['ret']
    X = df.drop(columns=['ret'])
    X = sm.add_constant(X)
    
    last_ts = df.index[-1]
    fc_date = next_working_days(last_ts, num_days=5)[horizon - 1]    

    forecast = pd.DataFrame({'date_time': [fc_date]})
    forecast.set_index('date_time', inplace=True)
    
    
    forecast['volatility'] = df.loc[:,'volatility'].iloc[-1]
    
    X_fc = forecast
    X_fc = sm.add_constant(X_fc, has_constant='add')

    quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
    model_qr = sm.QuantReg(y, X)

    for q in quantiles:
        model_temp = model_qr.fit(q=q)
        forecast_temp = model_temp.predict(X_fc)
        forecast[f'q{q}'] = forecast_temp

    fc_results = forecast.iloc[:,
                          forecast.columns.get_loc('q0.025'):forecast.columns.get_loc('q0.975') + 1]

    fc_results = fc_results.reset_index(drop=False)
    fc_results = fc_results.rename(columns={"date_time": "forecast_date"})
    fc_results['horizon'] = f'{horizon} day'
    fc_results['target'] = "DAX"
    fc_results = fc_results[
        ['forecast_date', 'target', 'horizon', 'q0.025', 'q0.25', 'q0.5', 'q0.75', 'q0.975']]
    
    return fc_results
    
    

In [10]:
quant_reg(df)
# df = df.dropna()
# _quant_reg_one_horizon(df['ret5'],0, 1)

  forecast_date target horizon    q0.025     q0.25      q0.5     q0.75    q0.975
0    2024-01-31    DAX   1 day -1.192141 -0.314656  0.058116  0.450124  1.161549
0    2024-02-01    DAX   2 day -1.461592 -0.460144  0.106265  0.672163  1.583035
0    2024-02-02    DAX   3 day -1.624667 -0.500691  0.183071  0.849433  1.816741
0    2024-02-05    DAX   4 day -1.881449 -0.485671  0.220067  1.002475  2.057018
0    2024-02-06    DAX   5 day -2.276466 -0.527292  0.398386  1.231128  2.298978
